# Consumo de agua CDMX

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data'
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' #####SÓLO TRAE INFO 2016
FILE_NAME = 'aborto_legal'

## Class

In [15]:
from pandas import DataFrame
from typing import Dict
from numpy import nan


from mariachis.models import BaseClass

from pandas import cut
from sklearn.preprocessing import OneHotEncoder

class AbortoLegal(BaseClass):
    def __init__(self, base_dir: str, file_name: str) -> None: 
        super().__init__(base_dir, file_name)

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

    def wrangling_aborto(self, df: DataFrame, clean_dict: Dict, date_col: str='fingreso', export_imputed: bool=True, **kwargs):
        # Sólo registros con fecha
        df = df[df[date_col].notnull()].reset_index()
        # Crear variables de fecha
        df = self.date_vars(df, date_col)

        # Obtener las variables numéricas
        vars_num = list(set(
            clean_dict['vars_numbin']+
            list(clean_dict['vars_num'].keys())
        ))
        
        # Limpiar si hay texto de variables numéricas
        for col in vars_num:
            df[col] = df[col].map(self.clean_number).astype(float)
        
        # Función para convertir 1.0 --> 01
        def two_char(n):
            return str(int(n)).zfill(2)

        # Crear rangos de variables numéricas
        for col, to_group in clean_dict['vars_num'].items():
            df[f'rango_{col}'] = cut(df[col], bins=[-1]+to_group+[1000])
            df[f'rango_{col}'] = df[f'rango_{col}'].map(lambda x: two_char(x.left+1)+' a '+two_char(x.right) if x!=nan else 'DESCONOCIDO')
            last_bin = two_char(to_group[-1]+1)
            df[[f'rango_{col}']] = df[[f'rango_{col}']].replace({
                **{f'{last_bin} a 1000':f'>{last_bin}'},
                **{two_char(x)+' a '+two_char(x):two_char(x) for x in to_group}
            })
            df[f'rango_{col}'] = df[f'rango_{col}'].map(lambda x: 'DESCONOCIDO' if str(x) in ('nan',nan) else str(x))
        
        # Sólo saber si son mayores a 0 o no
        for col in clean_dict['vars_numbin']:
            df[col] = df[col].map(lambda x: '>0' if x>0 else x)

        # Obtener las variables que serán binarias
        vars_cat = list(set(
            clean_dict['vars_first_word']+
            list(clean_dict['vars_cat'].keys())+
            clean_dict['vars_yes_no']+
            clean_dict['vars_just_fill_na']
        ))

        # Omitir acentos de variables categóricas
        for col in vars_cat:
            df[col] = df[col].map(self.clean_text)

        # Obtener la primer palabra
        for col in clean_dict['vars_first_word']:
            df[col] = df[col].str.split().str[0]

        # Agrupar categorías
        for col,to_group in clean_dict['vars_cat'].items():
            df[col] = df[col].map(to_group)

        cluster_cols = vars_cat+[f'rango_{col}' for col in clean_dict['vars_num'].keys()]+clean_dict['vars_numbin']
        
        # Lo que quedo vacío, marcar como "DESCONOCIDO"
        for col in cluster_cols:
            df[col] = df[col].fillna('DESCONOCIDO')

        # Tal vez el usuario quiere exportar los resultados
        if export_imputed: self.export_csv(df, name_suffix='imputed')
        if 1==1: return df[cluster_cols]

        X = OneHotEncoder().fit_transform(df[cluster_cols])
        if 1==1: return X
        X, cluster_pipe = self.make_clusters(X, cluster_cols=cluster_cols)
        if 1==1: return X

        # Estructurar la información limpia
        # df = df.pivot_table(index=['fingreso_yearmonth', 'entidad', 'alc_o_municipio'])
        # df.columns = ['_'.join([x for x in col]) if not isinstance(df.columns[0],str) else col for col in df.columns]
        return df

al = AbortoLegal(BASE_DIR, FILE_NAME)
# print(al)

## Data Wrangling

In [3]:
df = al.full_import(api=False)
df.sample()

,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
75698,2018.0,OCTUBRE,2018-10-01,NaN,SOLTERA,33.0,NINGUNO,PREPARATORIA,NaN,CATOLICA,...,NaN,NaN,NaN,NaN,COMPLETA,NaN,DFSSA003640,DFSSA003640,MEDICAMENTO,MISOPROSTOL + MIFEPRISTONA


## Dict

In [4]:
# Diccionario para aplicar limpieza por tipo de variable
full_dict = {}

# Ahora, limpiar texto de variables numéricas
full_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']
full_dict['vars_num'] = {
    'edad': [9,11,20,22,25,28,32],
    'menarca': [0,11,12,13,14],
    'fsexual': [0,15,16,17,18,19],
    'sememb': [0,5,6,7,8,9],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
full_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
full_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Sólo cambiar nulos por "DESCONOCIDO"
full_dict['vars_just_fill_na'] = ['entidad', 'motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep']

# Agrupar categorías
full_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA O MAYOR',
        'MAESTRIA':'LICENCIATURA O MAYOR',
        'DOCTORADO':'LICENCIATURA O MAYOR',
        'PREPARATORIA':'PREPARATORIA O MENOR',
        'SECUNDARIA':'PREPARATORIA O MENOR',
        'PRIMARIA':'PREPARATORIA O MENOR',
        'SIN ACCESO A LA EDUCACION FORMAL':'PREPARATORIA O MENOR',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

In [5]:
from pandas import qcut, cut

aux_dict = {}
for col in full_dict['vars_num']:
    new_col = df[col].map(al.clean_number).astype(float)
    orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
    new_bins = [orig_bins[0]-1] + list(orig_bins[:-1]) + [1e3]
    aux_dict[col] = new_bins

aux_dict

{'edad': [10.0, 11.0, 20.0, 22.0, 25.0, 28.0, 32.0, 1000.0],
 'menarca': [-1.0, 0.0, 11.0, 12.0, 13.0, 14.0, 1000.0],
 'fsexual': [-1.0, 0.0, 15.0, 16.0, 17.0, 18.0, 19.0, 1000.0],
 'sememb': [-1.0, 0.0, 5.0, 6.0, 7.0, 8.0, 9.0, 1000.0],
 'nhijos': [-1.0, 0.0, 1.0, 2.0, 1000.0],
 'gesta': [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 1000.0],
 'c_num': [-1.0, 0.0, 1.0, 2.0, 1000.0],
 'p_semgest': [-1.0, 0.0, 5.0, 6.0, 7.0, 8.0, 9.0, 1000.0],
 'p_diasgesta': [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 1000.0]}

## Transform

In [16]:
X = al.wrangling_aborto(df, full_dict)
X.sample(4)

,cconsejo,s_complica,edocivil_descripcion,motiles,nivel_edu,consejeria,desc_derechohab,panticoncep,autoref,desc_servicio,...,rango_sememb,rango_nhijos,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,naborto,npartos,ncesarea,nile
8501,SI,SI,SOLTERX,INTERRUPCION VOLUNTARIA,PREPARATORIA O MENOR,SI,NINGUNO,CONDON,NO,ILE,...,08,00,01,DESCONOCIDO,09,04,0.0,0.0,0.0,0.0
22854,SI,NO,SEPARADX,INTERRUPCION VOLUNTARIA,PREPARATORIA O MENOR,SI,ALGUNO,IMPLANTE,SI,CONSULTA EXTERNA,...,07,>03,04,DESCONOCIDO,07,DESCONOCIDO,0.0,>0,0.0,0.0
21727,SI,NO,SOLTERX,INTERRUPCION VOLUNTARIA,PREPARATORIA O MENOR,SI,DESCONOCIDO,DESCONOCIDO,NO,GINECO OBSTETRICIA,...,>10,DESCONOCIDO,01,00,>10,05,0.0,0.0,0.0,0.0
41330,SI,SI,SOLTERX,INTERRUPCION VOLUNTARIA,PREPARATORIA O MENOR,SI,NINGUNO,DESCONOCIDO,NO,DESCONOCIDO,...,09,01,02,02,09,04,0.0,>0,0.0,0.0


In [17]:
df.isnull().mean()

año                     0.000252
mes                     0.000252
fingreso                0.020818
autoref                 0.714986
edocivil_descripcion    0.012241
edad                    0.000252
desc_derechohab         0.009546
nivel_edu               0.008715
ocupacion               0.139718
religion                0.071295
parentesco              0.640064
entidad                 0.000403
alc_o_municipio         0.069960
menarca                 0.039356
fsexual                 0.047870
fmenstrua               0.139541
sememb                  0.078788
nhijos                  0.157853
gesta                   0.037656
naborto                 0.064317
npartos                 0.057655
ncesarea                0.063297
nile                    0.171555
consejeria              0.031838
anticonceptivo          0.118043
c_fecha                 0.361410
c_num                   0.366296
motiles                 0.014596
h_fingreso              0.838155
h_fegreso               0.945292
desc_servi

Siguientes pasos!
- numéricas a rangos, para definir a los nulos como nueva categoría
- clustering añomes+municipio con vars_cat+vars_catbin escaladas y calculando ['count','min','mean','max']
- perfilamiento
- agrupar por municipio, cómo es la distribución de los clústeres previos?
- Cómo vender lo que resuelve?